# Import packages

In [5]:
import pandas as pd
from teradataml import *
import ast
from teradataml.dataframe.copy_to import copy_to_sql

from modules.db import TeradataDatabase
from utils import clean_text, load_embedding_model
from constants import CLEANED_TEST_DATA_PATH, CLASS_EMBEDDINGS_PATH, PRODUCT_TEST_EMBEDDINGS_PATH, CLEANED_GPC_PATH, CLEANED_TEST_DATA_PATH, TEST_DATA_PATH, E5_LARGE_INSTRUCT_CONFIG_PATH

## Connect to database

In [6]:
td_db = TeradataDatabase()
td_db.connect()

### Insert Product Names in DB

In [7]:
df = pd.read_csv(CLEANED_TEST_DATA_PATH)
df.head()

,id,Item_Name,translated_text,cleaned_text
0,0,Americana Okra zero 400 gm,Americana Okra zero 400 gm,americana okra zero gm
1,1,ليمون اداليا 500 جم,Lemon Adalia 500 gm,lemon adalia gm
2,2,صلصه هاينز برطمان خصم عرض,Heinz Bartman Sauce Discount Offer,heinz bartman sauce discount offer
3,3,Dasani water 330ML,Dasani water 330ML,dasani water ml
4,4,بودرة عصير أناناس من سورس، 900 جم,"Soros Pineapple Juice Powder, 900g",soros pineapple juice powder g


In [8]:
df.rename(columns={"translated_text": "product_name"}, inplace=True)

In [9]:
df.drop_duplicates(subset=["product_name"], inplace=True)
df.dropna(subset=["product_name"], inplace=True)
df = df[["id" , "product_name"]]

In [10]:
copy_to_sql(df, "products", "demo_user", if_exists="replace")

### Insert Class Names in DB

In [11]:
df = pd.read_csv(TEST_DATA_PATH)
df.head()

,Item_Name,class,Brand,Weight,Number of units,Size of units,Price,T.Price,Pack,Unit
0,Americana Okra zero 400 gm,Vegetables & Fruits,Americana,400gm,1,NaN,NaN,NaN,كيس,gm
1,ليمون اداليا 500 جم,"Tins, Jars & Packets",NaN,500جم,1,NaN,NaN,NaN,علبة,جم
2,صلصه هاينز برطمان خصم عرض,"Tins, Jars & Packets",هاينز,NaN,1,NaN,NaN,NaN,علبة,NaN
3,Dasani water 330ML,Water,Dasani,330مل,1,NaN,NaN,NaN,زجاجة,مل
4,بودرة عصير أناناس من سورس، 900 جم,Soft Drinks & Juices,سورس,900جم,1,NaN,NaN,NaN,عبوة,جم


In [ ]:
df_class = df["class"].dropna().unique()
# df = df.reset_index(drop=False) 
# df.rename(columns={'index': 'id'}, inplace=True)
df_classes = pd.DataFrame({"class": df_class})

In [ ]:
df_classes["id"] = df_classes.index
df_classes = df_classes[["id", "class"]]

In [ ]:
copy_to_sql(df_classes, "classes", "demo_user", if_exists="replace")

In [16]:
df_actual_class = df[["class"]]
df_actual_class["product_id"] = df_actual_class.index

df_actual_class = df_actual_class[["product_id", "class"]]

In [18]:
copy_to_sql(df_actual_class, "actual_classes", "demo_user", if_exists="replace")

### Insert Product Embeddings in DB

In [ ]:
df = pd.read_csv(PRODUCT_TEST_EMBEDDINGS_PATH)

,id,embeddings
0,0,"[0.0131988525390625, 0.0203704833984375, -0.00..."
1,1,"[0.0016574859619140625, 0.041656494140625, 0.0..."
2,2,"[0.0162811279296875, 0.018951416015625, -0.025..."
3,3,"[0.0190277099609375, 0.0229949951171875, -0.01..."
4,4,"[0.0226593017578125, 0.0291900634765625, -0.02..."


In [ ]:
df['embeddings'] = df['embeddings'].apply(ast.literal_eval)
emb_cols = pd.DataFrame(df['embeddings'].tolist(), columns=[f'embed_{i}' for i in range(len(df['embeddings'][0]))])

df_expanded = pd.concat([df[['id']], emb_cols], axis=1)

In [35]:
copy_to_sql(df_expanded, "p_embeddings", "demo_user", if_exists="replace")

In [39]:
df_products = td_db.execute_query("Select * from demo_user.products")
df_products = DataFrame(df_products, False)

### Insert Class Embeddings in DB

In [ ]:
df = pd.read_csv(TEST_DATA_PATH)
df.head()

In [ ]:
df_class = df["class"].dropna().unique()
# df = df.reset_index(drop=False) 
# df.rename(columns={'index': 'id'}, inplace=True)
df = pd.DataFrame({"class": df_class})

In [14]:
model = load_embedding_model(E5_LARGE_INSTRUCT_CONFIG_PATH)

In [27]:
class_name = df["class"].tolist()
embeddings = model.get_embeddings(class_name)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
embeddings = embeddings.tolist()
emb_cols = pd.DataFrame(embeddings, columns=[f'embed_{i}' for i in range(len(embeddings[0]))])

In [29]:
df_expanded = pd.concat([df[['id']], emb_cols], axis=1)

In [32]:
copy_to_sql(df_expanded, "c_embeddings", "demo_user", if_exists="replace")

## IN-DB Similiraty

In [3]:
RESULT_TABLE = "demo_user.similiratiy_score"

In [35]:
# Create the result table
q = f"""
    CREATE TABLE {RESULT_TABLE} (
        item_id BIGINT,
        closest_category_id BIGINT,
        cosine_distance FLOAT
    );
    """

In [36]:
td_db.execute_query(q)

[]

In [43]:
# Generate the string of vector columns for 1024 dimensions
vector_cols = ", ".join([f"embed_{i}" for i in range(1024)])

# Generate the quoted string for feature columns
vector_cols_quoted = ", ".join([f"'embed_{i}'" for i in range(1024)])

In [ ]:
classification_sql = f"""
INSERT INTO {RESULT_TABLE}
WITH RankedDistances AS (
    SELECT
        o.Target_ID AS product_id,
        o.Reference_ID AS class_id,
        o.Distance,
        ROW_NUMBER() OVER (PARTITION BY o.Target_ID ORDER BY o.Distance ASC) as rn
    FROM TD_VectorDistance (
        ON (SELECT id, {vector_cols} FROM p_embeddings) AS TargetTable
        ON (SELECT id, {vector_cols} FROM c_embeddings) AS ReferenceTable DIMENSION
        USING
            TargetIDColumn('id')
            RefIDColumn('id')
            TargetFeatureColumns({vector_cols_quoted})
            RefFeatureColumns({vector_cols_quoted})
            DistanceMeasure('cosine')
    ) AS o
)
SELECT
    item_id,
    category_id,
    Distance
FROM RankedDistances
WHERE rn = 1;
"""
td_db.execute_query(classification_sql)

In [53]:
tdf = td_db.execute_query("SELECT * FROM demo_user.similiratiy_score")
tdf = DataFrame(tdf, False)
tdf

item_id,closest_category_id,cosine_distance
0,31,0.13978061328505798
11,31,0.13165661426991537
18,7,0.14913114673814587
1,31,0.1530110484109758
17,20,0.12880388116464037
23,1,0.15575709612568223
10,1,0.145510153212157
3,2,0.10794134237588049
8,28,0.15756675125841724
2,12,0.14420966831184534


In [4]:
results_query = f"""
SELECT
    p.product_name AS product_name,
    c."class" AS predicted_class,
    a."class" AS actual_class,
    r.cosine_distance AS similarity_score
FROM {RESULT_TABLE} r
JOIN products p
    ON r.item_id = p.id
JOIN classes c
    ON r.closest_category_id = c.id
JOIN actual_classes a
    ON a.product_id = p.id;
"""

In [5]:
tdf = td_db.execute_query(results_query)
df = pd.DataFrame(tdf)

In [6]:
df

,product_name,predicted_class,actual_class,similarity_score
0,Lemon Adalia 500 gm,"Jams, Spreads & Syrups","Tins, Jars & Packets",0.153011
1,Candlestick Chocolate Pure Hazelnut,"Chocolates, Sweets & Desserts",Biscuits & Cakes,0.114309
2,Lubya Baladi - 500 gm,"Jams, Spreads & Syrups","Rice, Pasta & Pulses",0.176968
3,Americana Okra zero 400 gm,"Jams, Spreads & Syrups",Vegetables & Fruits,0.139781
4,Dasani water 330ML,Water,Water,0.107941
...,...,...,...,...
4498,Mr Organic Pasta Sauce Italian Bolognese 350g,"Rice, Pasta & Pulses","Tins, Jars & Packets",0.138744
4499,Foody Fava Beans Plain 3 K,Vegetables & Fruits,"Tins, Jars & Packets",0.131579
4500,Saw chicken hips.,Poultry,Poultry,0.137450
4501,Almarai Whole Milk 1 L Plastic,Dairy & Eggs,Dairy & Eggs,0.145429


In [29]:
df.dropna(inplace=True)

In [30]:
from sklearn.metrics import f1_score

y_pred = df["predicted_class"].tolist()
y_true = df["actual_class"].tolist()

f1_score(y_true, y_pred, average="weighted")

0.4329434502414501

In [7]:
copy_to_sql(df, "results", "demo_user", if_exists="replace")

In [3]:
query = """
SELECT * FROM TD_ClassificationEvaluator (
   ON demo_user.results AS InputTable
   OUT PERMANENT TABLE OutputTable(classification_metrics)
   USING
       ObservationColumn('actual_class')
       PredictionColumn('predicted_class')
       Labels('Condiments, Dressings & Marinades', 'Furniture', 'Personal care, skin & body care', 'null', 'Tea, Coffee & Hot Drinks', 'Sweets & Desserts', 'Hair, Shower, Bath & Soap', 'Fruits', 'Nuts, Dates & Dried Fruits', 'Vegetables & Fruits', 'Home Appliances', 'Sauces, Dressings & Condiments', 'Baby Care', 'Tea and Coffee', 'Disposables & Napkins', 'Tins, Jars & Packets', 'Chips & Crackers', 'Soft Drinks & Juices', 'Cooking Ingredients', 'Dairy & Eggs', 'Bakery', 'Vegetables & Herbs', 'Biscuits & Cakes', 'Candles & Air Fresheners', 'Water', 'Rice, Pasta & Pulses', 'Poultry', 'Beef & Processed Meat', 'Home Textile', 'Cleaning Supplies', 'Beef & Lamb Meat', 'Chocolates, Sweets & Desserts', 'Jams, Spreads & Syrups')
) AS dt;
"""
tdf = td_db.execute_query(query)


In [4]:
DataFrame(td_db.execute_query("select * from demo_user.classification_metrics"))

SeqNum,Metric,MetricValue,index_label
3,Micro-Recall,0.41289023162134947,2
5,Macro-Precision,0.3528525087190308,4
6,Macro-Recall,0.322042812468733,5
7,Macro-F1,0.24408072902307343,6
9,Weighted-Recall,0.41289023162134947,8
10,Weighted-F1,0.4729390335334207,9
8,Weighted-Precision,0.6789587385301326,7
4,Micro-F1,0.41289023162134947,3
2,Micro-Precision,0.41289023162134947,1
1,Accuracy,0.41289023162134947,0


## Disconnect

In [ ]:
td_db.disconnect()